## Add packages

In [2]:
# import packages
import xarray as xr
import geopandas as gpd
import regionmask
import matplotlib.pyplot as plt
#import cartopy.crs as ccrs
import plotly.express as px


c:\Users\Kris\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


## Setup

In [3]:
# Define shape file location
shp_location = r"C:\Users\Kris\Documents\amazonforcast\gisfiles\hydrobasins shape\hybas_sa_lev05_areaofstudy.shp"
# Define netCDF file location
ds_location = r"C:\Users\Kris\Documents\amazonforcast\data\prakrut\output\LIS_HIST_Forecast_June_02_to_05.nc"

# .shp file loader
hybas_sa_lev05 = gpd.read_file(shp_location)
# netcdf file loader
ds = xr.open_dataset(ds_location)

# Reproject current crs of gdf to epsg 4326
print(f"East-West range: {ds.east_west.min().item()} to {ds.east_west.max().item()}")
print(f"North-South range: {ds.north_south.min().item()} to {ds.north_south.max().item()}")

print(hybas_sa_lev05.total_bounds)  # This will print the bounds of the regions in your GeoDataFrame

# Print current CRS of the GeoDataFrame
print(f"Original CRS: {hybas_sa_lev05.crs}")

# Reproject to WGS84 (latitude and longitude)
hybas_sa_lev05 = hybas_sa_lev05.to_crs(epsg=4326)

# Print new CRS to confirm
print(f"Reprojected CRS: {hybas_sa_lev05.crs}")

# Check the bounds again after reprojection
print(hybas_sa_lev05.total_bounds)

ds # get info about netcdf file


East-West range: -82.0 to -49.0
North-South range: -21.0 to 6.0
[-9054007.67266785 -2376978.16229946 -5440740.11254905   671070.13011205]
Original CRS: EPSG:3857
Reprojected CRS: EPSG:4326
[-81.33353475 -20.87498542 -48.875        6.01723395]


<xarray.Dataset>
Dimensions:           (time: 4, north_south: 540, east_west: 660, ensemble: 7,
                       SoilMoist_profiles: 4, SoilTemp_profiles: 4)
Coordinates:
  * time              (time) datetime64[ns] 2024-06-02 2024-06-03 ... 2024-06-05
  * ensemble          (ensemble) float32 1.0 2.0 3.0 4.0 5.0 6.0 7.0
  * north_south       (north_south) float64 -21.0 -20.95 -20.9 ... 5.9 5.95 6.0
  * east_west         (east_west) float64 -82.0 -81.95 -81.9 ... -49.05 -49.0
Dimensions without coordinates: SoilMoist_profiles, SoilTemp_profiles
Data variables: (12/32)
    lat               (time, north_south) float32 ...
    lon               (time, east_west) float32 ...
    Swnet_tavg        (time, ensemble, north_south, east_west) float32 ...
    Lwnet_tavg        (time, ensemble, north_south, east_west) float32 ...
    Qle_tavg          (time, ensemble, north_south, east_west) float32 ...
    Qh_tavg           (time, ensemble, north_south, east_west) float32 ...
    ...                ...
    Qair_f_tavg       (time, ensemble, north_south, east_west) float32 ...
    Psurf_f_tavg      (time, ensemble, north_south, east_west) float32 ...
    SWdown_f_tavg     (time, ensemble, north_south, east_west) float32 ...
    LWdown_f_tavg     (time, ensemble, north_south, east_west) float32 ...
    LAI_inst          (time, ensemble, north_south, east_west) float32 ...
    Greenness_inst    (time, ensemble, north_south, east_west) float32 ...
Attributes: (12/15)
    missing_value:           -9999.0
    NUM_SOIL_LAYERS:         4
    SOIL_LAYER_THICKNESSES:  [ 10.        30.000002  60.000004 100.      ]
    title:                   LIS land surface model output
    institution:             NASA GSFC
    source:                  
    ...                      ...
    comment:                 website: http://lis.gsfc.nasa.gov/
    MAP_PROJECTION:          EQUIDISTANT CYLINDRICAL
    SOUTH_WEST_CORNER_LAT:   -20.975
    SOUTH_WEST_CORNER_LON:   -81.975
    DX:                      0.05
    DY:                      0.05

In [4]:
aoi = hybas_sa_lev05[hybas_sa_lev05.PFAF_ID == 62297]
# Create a 3d mask - this contains the true / false values identifying pixels
# inside vs outside of the mask region
aoi_mask = regionmask.mask_3D_geopandas(aoi,
                                         ds.east_west,
                                         ds.north_south)
aoi_mask


<xarray.DataArray 'mask' (region: 1, north_south: 540, east_west: 660)>
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])
Coordinates:
  * north_south  (north_south) float64 -21.0 -20.95 -20.9 ... 5.9 5.95 6.0
  * east_west    (east_west) float64 -82.0 -81.95 -81.9 ... -49.1 -49.05 -49.0
  * region       (region) int64 90
Attributes:
    standard_name:  region

In [5]:
aoi_ds = ds['Rainf_tavg'].where(aoi_mask)
aoi_ds

<xarray.DataArray 'Rainf_tavg' (time: 4, ensemble: 7, north_south: 540,
                                east_west: 660, region: 1)>
array([[[[[nan],
          [nan],
          [nan],
          ...,
          [nan],
          [nan],
          [nan]],

         [[nan],
          [nan],
          [nan],
          ...,
          [nan],
          [nan],
          [nan]],

         [[nan],
          [nan],
          [nan],
          ...,
...
          ...,
          [nan],
          [nan],
          [nan]],

         [[nan],
          [nan],
          [nan],
          ...,
          [nan],
          [nan],
          [nan]],

         [[nan],
          [nan],
          [nan],
          ...,
          [nan],
          [nan],
          [nan]]]]], dtype=float32)
Coordinates:
  * time         (time) datetime64[ns] 2024-06-02 2024-06-03 ... 2024-06-05
  * ensemble     (ensemble) float32 1.0 2.0 3.0 4.0 5.0 6.0 7.0
  * north_south  (north_south) float64 -21.0 -20.95 -20.9 ... 5.9 5.95 6.0
  * east_west    (east_west) float64 -82.0 -81.95 -81.9 ... -49.1 -49.05 -49.0
  * region       (region) int64 90
Attributes:
    units:          kg m-2 s-1
    standard_name:  precipitation_rate
    long_name:      precipitation rate
    vmin:           0.0
    vmax:           0.0

In [43]:
summary = aoi_ds.groupby("time").mean(["north_south", "east_west"])
aoi_ds_summary = summary.to_dataframe().reset_index()
aoi_ds_summary

,time,ensemble,region,Rainf_tavg
0,2024-06-02,1.0,90,0.000044
1,2024-06-02,2.0,90,0.000044
2,2024-06-02,3.0,90,0.000045
3,2024-06-02,4.0,90,0.000045
4,2024-06-02,5.0,90,0.000043
5,2024-06-02,6.0,90,0.000044
6,2024-06-02,7.0,90,0.000043
7,2024-06-03,1.0,90,0.000055
8,2024-06-03,2.0,90,0.000055
9,2024-06-03,3.0,90,0.000056


## Building time series with spread using boxplot

In [53]:

fig = px.box(aoi_ds_summary, y = 'Rainf_tavg', x = 'time', color='time')
fig.update_layout(title='Precipitation of Region (PFAF-ID) 62297', height = 800, width = 800)
fig.show()

